# Handling shapefiles in Python

<span style="color:red">Shapefiles</span> are one of the most popular file formats for storing vector Geo Spatial data. Shapefiles have a file extension of `.shp` The shapefile was created by Esri, a software vendor in the early 1990s. You can take a deep dive into the whitepaper __[here](https://www.esri.com/library/whitepapers/pdfs/shapefile.pdf)__.

__[GADM](https://gadm.org/download_country_v3.html)__ is a great source from where you can download shapefiles for the country of your choice. I will be using the shapefiles of Switzerland for this example.

After downloading the `.zip` file and extracting the contents, you should note two important things.
1. The data are arranged into a hierarchy. The filenames end with <span style="color:blue">**$adm0$**</span>, <span style="color:blue">**$adm1$**</span> or <span style="color:blue">**$admn$**</span>. They indicate the levels of __administrative regions__ of the country. **$adm0$** indicates that the shapefile contains GIS data for the entire country. **$adm1$** indicates that the shapefile contains GIS data for the states and so on till a finer level of granularity is reached. The Swiss dataset only has administrative levels up to **$adm1$**. Other datasets might have finer levels of granularity. When looking for geo spatial data, always ensure that you have the correct granularity of the data. If you are working on mapping public transport routes a shapefile containing town/city level granularity might better suit your needs than a shapefile with county/district granularity.
2. Apart from the `.shp` file there are files bearing the same name and having different extensions. Let us look at what they signify. 
    * `.shp` - This is the main data file. It is a variable-record-length file in which each record describes a shape with a list of its vertices.
    * `.shx` - This is the Index file. Each record contains the offset of the corresponding main file record from the beginning of the main file.
    * `.dbf` - This is the dBASE Table file. DBF contains feature attributes with one record per feature. The one-to-one relationship between geometry and attributes is based on record number. Attribute records in the dBASE file must be in the same order as records in the main file.
    * `.cpg` - An optional file that can be used to specify the codepage for identifying the character set to be used.
    * `.prj` - Projections Definition file; stores coordinate system information.
    
The `.shp`, `.shx` and `.prj` files must always be in the same directory structure. Failing that would make a singular shapefile unreadable as we would lose the index data along with the record locators for geo spatial features.

There are plenty of ways to view the contents of a shapefile. The quickest and easiest way to do so is to use __[QGIS]( https://qgis.org/en/site/forusers/download.html)__, a powerful GIS mapping software. However, I'm going to show you how to view a shapefile in Python using the excellent `mplleafet` library.